In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
import optuna.integration.lightgbm as lgbo

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error # 平均絶対誤差
#from sklearn.metrics import mean_squared_error # 平均二乗誤差
#from sklearn.metrics import mean_squared_log_error # 対数平均二乗誤差
from sklearn.metrics import r2_score # 決定係数
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import missingno as msno
import plotly.express as px

import json
from collections import OrderedDict

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# 1. Process
(1) Adjust the parameters using the data extracted from the rows that do not contain missing values.<br>
(2) Create a model by training the columns containing missing values as objective variables in order.(F_1_n:15types,F_3_n:25types,F_4_n:15types,Total 55types)<br>
(3) Collect the data containing missing values in the same column, pass it to the model created in (2), make a prediction, and complement "data".

# 2. Import data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv")
data = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/data.csv")

In [ ]:
sample_submission

In [ ]:
data

In [ ]:
data.info()

There are no missing values in F_2_n.<br>
F_2_ｎには欠損値がありません。

# 3. EDA

In [ ]:

# 欠損値の分布を可視化
# Search for missing data

msno.matrix(df=data, figsize=(10,6), color=(0,.3,.3))


In [ ]:

# 特徴量ごとの分布を確認
# Check the distribution for each feature

data.drop(columns=['row_id']).describe().T\
        .style.bar(subset=['mean'], color=px.colors.qualitative.G10[0])\
        .background_gradient(subset=['std'], cmap='Greens')\
        .background_gradient(subset=['50%'], cmap='BuGn')


In [ ]:

# 特徴量ごとの分布を可視化
# Visualization of distribution for each feature

figure = plt.figure(figsize=(16, 18))
count = 0
for i in range(4):
    if(i == 0 or i == 3):
        featCount = 15
    else:
        featCount = 25
    for j in range(featCount):
        feat_name = f'F_{i+1}_{j}'
        plt.subplot(16, 5, count+1)
        plt.hist(data[feat_name], bins=100)
        plt.title(f'{feat_name}')
        count += 1
figure.tight_layout(h_pad=1.0, w_pad=0.8)
plt.show()


F_1_nとF_3_nとF_4_nは、ほぼ正規分布していますが、一部外れ値もありそうです。<br>
F_2_nは0以上の整数値をとっているようです。<br>
とりあえず、外れ値の置き換え等を行わずに処理を進めます。<br>
F_1_n, F_3_n, and F_4_n are almost normally distributed, but there are some outliers.<br>
F_2_n seems to take an integer value greater than or equal to 0.<br>
For the time being, proceed with processing without replacing outliers.

In [ ]:

# 特徴量間の相関を可視化
# Heatmap

corr = data.corr().round(2)
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()


特徴量同士の相関関係はほとんどありませんが、F_2_ｎ同士とF_4_ｎ同士では若干の相関関係がみられます。<br>
There is almost no correlation between features, but there is some correlation between F_2_ｎ and between F_4_ｎ.

In [ ]:
# 特徴量ごとの欠損値の数を確認
# Check the number of missing values for each feature

data.drop(columns='row_id').isnull().sum(axis=0)

In [ ]:
# idごとの欠損値の数を確認
# Check the number of missing values for each id

data.isnull().sum(axis=1)

複数の欠損値を持つ行が存在します<br>
There are rows with multiple missing values

# 4. Feature engineering

In [ ]:
# "data"から特徴量名を取り出す
# Extract feature name from "data"
allCols = data.drop(columns='row_id').columns
colAry = []
for i in range(len(allCols)):
    inner = allCols[i]
    if inner.split('_')[1] != '2':
        colAry.append(inner)
colAry

In [ ]:
# 外れ値を置き換える
# Replace outliers

replacedData = data.copy()
def replace_outliers(df,stds):
    outValue = stds * 3
    replaceValuePlus = df.median() + outValue#np.nan、df.mean()、df.median()
    replaceValueMinus = df.median() - outValue
    df[(df - df.median()) > outValue] = replaceValuePlus
    df[(df.median() - df) > outValue] = replaceValueMinus
    return df

for i in range(len(colAry)):
    colName = colAry[i]
    colData = replacedData[colName].copy()
    std = colData.std()
    replacedData[colName] = replace_outliers(colData,std)
    
replacedData

In [ ]:
# add columns
# Number of missing values
#replacedData['nullCount'] = replacedData.isnull().sum(axis=1)
#replacedData

# 5. Parameters

In [ ]:
# パラメータチューニング用に欠損値を含まない行を抜き出す
# Extract rows that do not contain missing values for parameter tuning

dataForOptuna = replacedData[replacedData.notnull().all(axis=1)]
#dataForOptuna = replacedData.copy()
dataForOptuna

In [ ]:

# optunaでパラメータを探索・保存
# Save parameters using optuna

paramObj = {}
subOptAry = []
for i in range(len(colAry)):
    if i >= 0:
        target = colAry[i]
        print('')
        print('***** Target : {} *****'.format(target))

        category = int(target.split('_')[1])
        num = int(target.split('_')[2])
        if category == 1:
            #F_1_n
            opt_rounds = 10
            opt_stop = None
        elif category == 3:
            #F_3_n
            opt_rounds = 10
            opt_stop = None
        elif category == 4:
            #F_4_n
            opt_rounds = 10#200
            opt_stop = None#100
    #    else:
    #        opt_rounds = 100
    #        opt_stop = None


        X = dataForOptuna.drop(columns=['row_id', target])
        value = dataForOptuna[target]

        X_train, X_test, t_train, t_test = train_test_split(X, value, test_size=0.2, random_state=0)

        opt_params = {
            "objective":"regression",
            "metric":"rmse"
        }

        X_train,X_test,y_train,y_test = train_test_split(
            X,
            value,
            test_size=0.2
        )

        reg_train = lgb.Dataset(
            X_train,
            y_train
        )

        reg_eval = lgb.Dataset(
            X_test,
            y_test,
            reference=reg_train
        )

        opt=lgbo.train(
            opt_params,
            reg_train,
            valid_sets = reg_eval,
            verbose_eval=False,
            num_boost_round = opt_rounds,
            early_stopping_rounds = opt_stop
        )

        paramObj[target] = opt.params
        subOptAry.append(opt.params)

        print(opt.params)

        # Save parameter objects on case 
        if (category == 1 and num == 14) or (category == 3 and num == 24):
            # Case of 'F_1_n' or 'F_3_n'
            with open('/kaggle/working/F_' + str(category) + '.json', 'w') as path:
                json.dump(subOptAry, path, indent=4)
            subOptAry = []
        elif (category == 4):
            # Case of 'F_4_n'
            with open('/kaggle/working/' + target + '.json', 'w') as path:
                json.dump(opt.params, path, indent=4)


In [ ]:
'''
# optunaを使ってパラメータを保存した後の処理
# Processing after saving parameters using optuna

F_1_json = pd.read_json("/kaggle/input/tps-jun-2022/F_1.json")
F_3_json = pd.read_json("/kaggle/input/tps-jun-2022/F_3.json")
F_4_0_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_1_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_2_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_3_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_4_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_5_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_6_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_7_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_8_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_9_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_10_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_11_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_12_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_13_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")
F_4_14_json = pd.read_json("/kaggle/input/tps-jun-2022/F_4_0.json", orient="index")

paramObj = {}
for i in range(len(colAry)):
    key = colAry[i]
    if i < 15:
        for j in range(0, 15):
            if type(F_1_json[key][j]) == dict:
                value = F_1_json[key][j]
                break
    elif 15 <= i < 40:
        for j in range(0, 25):
            if type(F_3_json[key][j]) == dict:
                value = F_3_json[key][j]
                break
    elif i >= 40:
        value = eval(key + '_json')[0]
    paramObj[key] = value
#paramObj
'''

# 6. Modering and prediction

In [ ]:
%%time

for i in range(len(colAry)):
    target = colAry[i]
    params = paramObj[target]
    print('')
    print('***** Target : {} *****'.format(target))
    print('----- Params --------------------------------------------------------------')
    print(params)
    print('----- Result --------------------------------------------------------------')
    print('Count : {} / 55'.format(i + 1))
    
    category = int(target.split('_')[1])
    if category == 1:
        #F_1_n
        learning_rate = 0.0001#0.0001
        num_iterations = 5000#5000
        e_s_rounds=100
    elif category == 3:
        #F_3_n
        learning_rate = 0.001#0.001
        num_iterations = 5000#5000
        e_s_rounds=100
    elif category == 4:
        #F_4_n
        learning_rate = 0.2#0.2
        num_iterations = 10000#10000
        e_s_rounds=100
#    else:
#        learning_rate = 0.1
#        num_iterations = 100
#        e_s_rounds=100
    
#-----------------------------------------------------------
    X = replacedData.drop(columns=['row_id', target])
    value = replacedData[target]
#-----------------------------------------------------------

    X_train, X_test, t_train, t_test = train_test_split(X, value, test_size=0.2, random_state=0)

    lgb_train = lgb.Dataset(X_train, t_train)
    lgb_eval = lgb.Dataset(X_test, t_test, reference=lgb_train)
    
    params = {
        'task': 'train',
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': learning_rate,
        'lambda_l1': paramObj[target]['lambda_l1'],
        'lambda_l2': paramObj[target]['lambda_l2'],
        'num_leaves': paramObj[target]['num_leaves'],
        'feature_fraction': paramObj[target]['feature_fraction'],
        'bagging_fraction': paramObj[target]['bagging_fraction'],
        'bagging_freq': paramObj[target]['bagging_freq'],
        'min_child_samples': paramObj[target]['min_child_samples'],
        'num_iterations': num_iterations,
        'verbosity': -1
    }

    model = lgb.train(
        params,
        train_set=lgb_train,
        valid_sets=lgb_eval,
        early_stopping_rounds=e_s_rounds,
        verbose_eval=1000
    )
    
    data_null = pd.DataFrame(replacedData[replacedData[target].isnull()])
    tmpAry = data_null.copy()
    X_test = data_null.drop(columns=['row_id', target])
    tmpAry[target] = model.predict(X_test)

    data.update(tmpAry)

In [ ]:
%%time
for i in range(len(sample_submission)):
    id = sample_submission['row-col'][i]
    row_id = id.split('-')[0]
    colName = id.split('-')[1]
    value = data[colName][int(row_id)]
    sample_submission['value'][i] = value
sample_submission

# 7. Make submission file

In [ ]:
sample_submission.to_csv('submission.csv', index=False)